In [1]:
import torch
import numpy as np

class StandardScaler():
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
        
    def fit_transform(self, data):
        self.mean = data.mean()
        self.std = data.std()
        
        return (data - self.mean) / self.std
    
    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean

/home/cseadmin/dz/anaconda3/envs/torch1.11/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [124]:
# np.random.seed(666) # 64不能过 32能过
np.random.seed(233) # 64能过 32不能过
# np.random.seed(2333) # 64和32都能过

samples=1280
arr=np.zeros(samples)
ran=np.random.random(samples*100)#*100

arr=np.concatenate((arr, ran))

scaler = StandardScaler(
    mean=arr.mean(), std=arr.std()
)

print((arr==0).sum())
print(arr.dtype)

1280
float64


In [125]:
def test_numpy(arr, scaler):
    arr_tf = scaler.transform(arr)
    res = scaler.inverse_transform(arr_tf)
    print((res==0).sum())
    print(res.dtype)

test_numpy(arr, scaler)

1280
float64


In [92]:
def test_tensor_numpy(arr, scaler):
    arr_tf = scaler.transform(arr)
    arr_tensor=torch.FloatTensor(arr_tf)
    arr_tensor_numpy=arr_tensor.numpy()
    
    res = scaler.inverse_transform(arr_tensor_numpy)
    print((res==0).sum())
    print(res.dtype)

test_tensor_numpy(arr, scaler)

0
float32


In [5]:
def test_tensor_from_numpy_to_numpy(arr, scaler):
    arr_tf = scaler.transform(arr)
    arr_tensor_from_numpy=torch.from_numpy(arr_tf)#.float()
    arr_tensor_from_numpy_to_numpy=arr_tensor_from_numpy.numpy()

    res = scaler.inverse_transform(arr_tensor_from_numpy_to_numpy)
    
    print((res==0).sum())
    print(res.dtype)

test_tensor_from_numpy_to_numpy(arr, scaler)

1280
float64


In [6]:
def test_tensor_cuda_numpy(arr, scaler):
    arr_tf = scaler.transform(arr)
    arr_tensor_cuda=torch.FloatTensor(arr_tf).cuda()
    arr_tensor_cuda_numpy=arr_tensor_cuda.cpu().numpy()

    res = scaler.inverse_transform(arr_tensor_cuda_numpy)
    print((res==0).sum())
    print(res.dtype)
    
test_tensor_cuda_numpy(arr, scaler)

0
float32


In [7]:
def test_dataloader(arr, scaler):
    arr_tf = scaler.transform(arr)
    arr_tensor=torch.FloatTensor(arr_tf)
    dataset = torch.utils.data.TensorDataset(arr_tensor, arr_tensor)
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=64, shuffle=False
    )

    all=[]
    for batch, _ in loader:
        batch = batch.cuda()

        batch = batch.cpu().numpy()
        all.append(batch)
        
    all = np.vstack(all).squeeze()

    res = scaler.inverse_transform(all)
    print((res==0).sum())
    print(res.dtype)
    
test_dataloader(arr, scaler)

0
float32


In [8]:
import os

def gen_xy(data, in_steps, out_steps, with_embeddings=False):
    if data.ndim == 2:
        data = data[:, :, np.newaxis]

    all_steps = data.shape[0]
    indices = [
        (i, i + (in_steps + out_steps))
        for i in range(all_steps - (in_steps + out_steps) + 1)
    ]

    x, y = [], []
    for begin, end in indices:
        x.append(data[begin : begin + in_steps])
        y.append(data[begin + in_steps : end])

    x = np.array(x)
    y = np.array(y)

    if with_embeddings:
        y = y[..., 0][..., np.newaxis]

    return x, y

train_size=0.7
val_size=0.1

data_path="./data/METRLA"
dataset="METRLA"

data = np.load(os.path.join(data_path, f"{dataset}.npy")) # equivalent to read_hdf(xxx).values
data = data[:, :, np.newaxis]

all_steps = data.shape[0]
split1 = int(all_steps * train_size)
split2 = int(all_steps * (train_size + val_size))

train_data = data[:split1]
val_data = data[split1:split2]
test_data = data[split2:]

x_train, y_train = gen_xy(train_data, 12, 12, False)
x_val, y_val = gen_xy(val_data, 12, 12, False)
x_test, y_test = gen_xy(test_data, 12, 12, False)

print(y_test.shape)
print(y_test.dtype)

(6832, 12, 207, 1)
float64


In [9]:
y_test_npz=np.load(os.path.join(data_path, "test.npz"))["y"][..., :1]

print(y_test_npz.shape)
print(y_test_npz.dtype)

(6850, 12, 207, 1)
float64


In [10]:
print((y_test==0).sum())
print((y_test_npz==0).sum())

2060124
2060359


In [11]:
scaler2 = StandardScaler(
    mean=x_train[..., 0].mean(), std=x_train[..., 0].std()
)

In [12]:
test_numpy(y_test, scaler2)
test_numpy(y_test_npz, scaler2)

2060124
float64
2060359
float64


In [13]:
test_tensor_numpy(y_test, scaler2)
test_tensor_numpy(y_test_npz, scaler2)

0
float32
0
float32


In [14]:
test_tensor_from_numpy_to_numpy(y_test, scaler2)
test_tensor_from_numpy_to_numpy(y_test_npz, scaler2)

2060124
float64
2060359
float64


In [15]:
test_tensor_cuda_numpy(y_test, scaler2)
test_tensor_cuda_numpy(y_test_npz, scaler2)

0
float32
0
float32


In [16]:
test_dataloader(y_test, scaler2)
test_dataloader(y_test_npz, scaler2)

0
float32
0
float32


In [17]:
print(torch.from_numpy(np.zeros(1)).dtype)
print(torch.from_numpy(np.zeros(1)).float().dtype)

torch.float64
torch.float32


### 关于训练模型时 StandardScaler.inverse_transform() 的调查报告

*23.3.26 更新：float32 个锤子，重点是别 transform y_true !!!!!!!!!!!!!!!!!! 往下看往下看往下看有新的实验*

#### 现象

0 transform 过去，变成 tensor, 然后 numpy() 变回 ndarray 之后, 再 inverse_transform 回来, 就不是 0 而是一个很小的数

y_true 在分母上，这一套下来原本的 0 值变成了一个超小的数，然后就会导致 MAPE 爆炸

#### 原因

torch 中的 tensor 为 float32 类型，而原始 numpy array 为 float64 类型

无论是否在 cuda 上，在 numpy -> tensor -> numpy 的过程中，实际上执行了强制向下转型，相当于 double 强转 float，损失精度

#### 证明

* 看以上的 5 个 test
    1. 就用原始 numpy
    2. numpy transform 之后 转 tensor 再转回 numpy
    3. numpy transform 之后 使用 torch.from_numpy() 转 tensor 再转回 numpy
    4. numpy transform 之后 转 tensor 放到 cuda 上再放回 cpu 上 再转回 numpy
    5. 模拟训练时的真实 dataloader

    返回值为结果中 0 值的个数以及它的 dtype

* 三组数据:
    1. np.zeros(samples)
    2. 我自己生成的 LA 的 y_test
    3. generate_training_data.py 生成的 test.npz 中的 y

* 期望的结果:

    结果中 0 值的个数和一开始读取进来的原数据中 0 值个数相等

* 错误结果：

    结果中 0 值的个数为 0
    
* 结果

    数据集 2，3 的 test 2 4 5 全挂，数据 1 玄学

    数据 2，3 是可以稳定复现的，数据 1 因为牵扯到 random，会出现各种情况<br>
    例如在上面的代码中我列出了 3 个 seed，分别能跑出三种不同的情况<br>
    ```python
    np.random.seed(666) # 64不能过 32能过
    np.random.seed(233) # 64能过 32不能过
    np.random.seed(2333) # 64和32都能过
    ```
    但是，当把 random 结果乘 100，也就是把随机数生成区间从 0~1 扩到 0~100 之后，这 5 个 test 就都能过了，猜测可能是一些超小的随机数在计算过程中 underflow 了<br>
    不过也不能证明放大之后就一定行，也可能是我没找到特定的 seed

    总之，数据1的结果涉及 random 的一些玄学，看来不可靠. 不过不重要，因为我们有可复现的两组确定的值

* 补充

    实际上数据3的结果也不算完全严谨，因为它和数据2用的是同一个scaler. 数据3在真实使用的过程中是没有任何问题的.<br>虽然都是0.2的测试集，但是2和3还是有一些小差异，结合上面的random玄学，或许就是这一点点差异导致了数据3在使用中没有碰到问题，但是并不代表问题不存在.<br>不过这并不影响这个实验结果的可信性，因为 scaler 本质就是一个线性变换和反变换，即使我把 mean 和 std 都设成随机数，也不会影响实验结论，因为问题本质是向下转型导致的精度丢失

#### 特例

`torch.from_numpy(arr)` 会创建 float64 类型的 tensor，然而没法用，因为模型里的参数都是 float32 类型的，不支持运算

所以就会有 `torch.from_numpy(arr).float()` 这个写法了，`.float()` 的作用就是强转 float32

综上，`torch.Tensor(arr) == torch.FloatTensor(arr) == torch.from_numpy(arr).float()` 这三者的返回值是一样的

#### 结论

1. 干脆别 transform y_true 就没这些破事了

2. 当然还有一种选择是在读取原始 numpy 数组的时候就 `.astype(np.float32)` (已证明可行)，例如 `read_hdf(path).values.astype(np.float32)`<br>反正变 tensor 都要强转，不如从头开始转好，保证整个 data flow 中类型的一致性？

再重新梳理一下整个过程，np.float64 的 0 先是被 transform 成了某个 np.float64 的值 m，然后在它转 tensor 的过程中变成了 torch.float32 类型，损失了精度. 因此，m 在 inverse_transform 的过程中很可能无法 map 回一个 np.float32 的 0. 然而，如果在开头就把这个 0 转成一个 np.float32 类型的 0，那么它 transform 成的 m 也就不会遭遇向下转型的问题，这个 m 一定可以 map 回一个 np.float32 的 0，和开头一致.

对于方法1，虽然没有影响，但是 x 在 transform 之后转 tensor 的过程中也会客观存在向下转型的问题，只不过是不影响 y_pred

因此，我认为方法2更能从本质上解决此问题

当然，方法1+2应该更无懈可击

#### 不要碰 sklearn 的 Scaler

`sklearn.preprocessing.StandardScaler` 这位更是重量级，他连最基本的 test1 都过不了，下面请欣赏:

In [108]:
import sklearn.preprocessing

trash_scaler=sklearn.preprocessing.StandardScaler()

np.random.seed(888)
arr=np.zeros((10, 10))
ran=np.random.random((10, 10))*1000 # 这里使劲放大 避免生成一些特别接近 0 的小数导致 underflow

arr=np.concatenate((arr, ran))
trash_scaler.fit(arr)

trash_scaler.inverse_transform(trash_scaler.transform(arr))[:10] # 此结果可复现性跟 seed 有关

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.84217094e-14, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.84217094e-14, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.84217094e-14, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.84217094e-14, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.84217094e-14, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
   

In [106]:
scaler3=StandardScaler(mean=arr.mean(), std=arr.std())
scaler3.inverse_transform(scaler3.transform(arr))[:10] # 正常结果

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [109]:
ran # 没有很小的数，不存在 underflow

array([[859.56060606, 164.56949536, 483.47595916, 921.0272662 ,
        428.55644132,  57.46009258, 925.00743443, 657.60153838,
        132.95283812, 533.44892506],
       [899.47760247, 248.36495841,  30.17181961,  72.44714713,
        874.16449484, 558.43035064, 916.0473573 , 633.46044603,
        283.25261043, 365.36880957],
       [ 92.23385593, 372.51258294, 347.42277894, 705.17076602,
        648.50903915,  40.90877305, 211.73176489,   1.48992314,
        138.97165701, 211.82539406],
       [ 26.09493297, 446.08735315, 239.1053078 , 954.4922192 ,
        907.63182395, 862.49050287,  91.58743709, 977.45235499,
        411.50139009, 458.30466961],
       [525.90924761, 294.41554361, 972.11594431, 181.44419861,
        303.40641894, 174.45413484, 527.56957655,  20.6929621 ,
         63.54593412, 635.27231208],
       [496.20334782,  14.1263981 , 627.22219182, 634.97506577,
        108.14148996, 829.64259705, 517.75216617, 570.68344442,
        546.33304626, 127.14921061],
       [72

In [111]:
print(arr.mean())
print(arr.std())

239.5926788373776
319.2670582859939


In [3]:
lin=torch.nn.Linear(8, 16)

lin.weight.dtype

torch.float32

In [12]:
lstm=torch.nn.LSTM(8, 16)

lstm.weight_hh_l0.dtype

torch.float32

In [6]:
emb=torch.nn.Embedding(10, 8)

emb.weight.dtype

torch.float32

In [7]:
randn=torch.randn(10, 10)

randn.dtype

torch.float32

---

## 2023.3.26 更新

出大问题，貌似我上面做的关于 random 的实验，那三个种子三种不同情况是真的。0 transform 之后能不能 inverse 回 0 真就是玄学，和 float32 还是 64 无关。

具体看下面 PEMS03 这个数据集，明明我转了 float32，但是在 numpy 这一步就挂了。。还没转 tensor 呢 没有什么强转丢精度问题。

这样就体现出方法 1 的重要性了，千万别 transform y_true，就一定不会碰见这些恶心问题了。

以及，我还是建议 `astype(float32)`，消除丢精度的问题（虽然现在看来没这么重要了。。）

In [29]:
pems03=np.load("./data/PEMS03/PEMS03.npz")["data"].astype(np.float32)
print(pems03.min())

scaler4=StandardScaler(mean=pems03.mean(), std=pems03.std())

trans03=scaler4.transform(pems03)
inverse03=scaler4.inverse_transform(trans03)

inverse03.min()

0.0


1.5258789e-05

In [28]:
tensor03=torch.FloatTensor(trans03)
tensor03_tonp=tensor03.numpy()
print(tensor03_tonp.dtype)
tensor03_tonp_inverse=scaler4.inverse_transform(tensor03_tonp)

tensor03_tonp_inverse.min()

float32


1.5258789e-05

In [16]:
trans03[pems03==0]

array([-1.2473868, -1.2473868, -1.2473868, ..., -1.2473868, -1.2473868,
       -1.2473868], dtype=float32)

In [9]:
inverse03[pems03==0]

array([1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 1.5258789e-05,
       1.5258789e-05, 1.5258789e-05], dtype=float32)

In [19]:
scaler4.transform(np.float32(0))

-1.2473868

In [20]:
scaler4.inverse_transform(np.float32(-1.2473868))

1.5258789e-05

In [17]:
trans03[pems03==0][0]

-1.2473868

In [26]:
pems03=np.load("./data/PEMS03/PEMS03.npz")["data"].astype(np.float64)
scaler4=StandardScaler(mean=pems03.mean(), std=pems03.std())

trans03=scaler4.transform(pems03)
inverse03=scaler4.inverse_transform(trans03)

inverse03.min()

-2.842170943040401e-14

如何解决 PEMS03 的这个问题：原数据保持 float64，mean 和 std 用 float32 算，然后莫名其妙就行了

按照之前的尿性，这个方法铁定也是玄学，没有普适性

In [25]:
pems03=np.load("./data/PEMS03/PEMS03.npz")["data"].astype(np.float64)
scaler4=StandardScaler(mean=pems03.astype(np.float32).mean(), std=pems03.astype(np.float32).std())

trans03=scaler4.transform(pems03)
inverse03=scaler4.inverse_transform(trans03)

inverse03.min()

0.0